In this file we implement a prediction algorithm based on AA-CBR, as introduced in Cyras, K., Satoh, K., & Toni, F. (2016, March). Abstract argumentation for case-based reasoning. In Fifteenth International Conference on the Principles of Knowledge Representation and Reasoning.

In [5]:
import pandas as pd
import import_ipynb

# the Mushroom data set is prepared in another file
import Data_preparation as d

# the objects Case, Feature and CB are defined in another file
from Classes import Case, Feature, CB

# selection of important features
important_mushroom = ['odor_a', 'odor_c', 'odor_f', 'odor_l', 'odor_n', 'odor_p', 'gill-size_b', 'gill-size_n', 
        'gill-color_b', 'stalk-surface-above-ring_k', 'stalk-surface-below-ring_y', 'ring-type_f', 'spore-print-color_k', 
                      'spore-print-color_n', 'spore-print-color_r', 'spore-print-color_u', 'population_c']

def create_cb():
    
    # import data, name of the outcome variables and list of feature correlations with the outcome variable
    data, y_name, _,_ = d.get_mushroom()
    # we take some subset, to speed up
    data = data.iloc[0:500,:]
    
    # we create a list of all features, consisting of all columns that are important and not the outcome variable
    features = []
    for col in data.columns:
        if col != y_name:
            if col in important_mushroom:
                features.append(Feature(col, None, None))
                
    # we create a case base
    cb = []
    for i, row in data.iterrows():
        outcome = row[y_name]
        f_s = {}
        
        for f in features:
            # only features that are present (value 1) are added
            if row[f.name] > 0:
                f_s[f] = row[f.name]
                            
        # we define as case as a triple (name, fact situation, outcome)
        cb.append(Case(i, f_s, outcome))
        
    return(CB(cb))


Let the system make a prediction by finding out whether the default outcome is part of the grounded extension

In [6]:

# receives an argument A and some cb
# returns all arguments in the cb that are attacked by A
def are_attacked(attacker, cb):
    more_specific = []
    for case in cb:
        if attacker.outcome != case.outcome:
            if set(attacker.fs.keys()) > set(case.fs.keys()):
                more_specific.append(case)
    
    removals = []
    for case in more_specific:
        for other_attacker in cb:
            if set(attacker.fs.keys()) > set(other_attacker.fs.keys()):
                if set(other_attacker.fs.keys()) > case.fs.keys():
                    removals.append(case)
                    
    attacked = [a for a in more_specific if a not in removals]
    return attacked
                
# receives an argument A and some cb
# returns whether A is unattacked by the cb (True/False)
def is_unattacked(argument, cb):
    for case in cb:
        if argument.outcome != case.outcome:
            if set(argument.fs.keys()) < set(case.fs.keys()):
                return False
    return True

# receives some cb
# returns all arguments in the cb that are unattacked
def get_unattacked(cb):
    unattacked = []
    for argument in cb:
        if is_unattacked(argument, cb):
            unattacked.append(argument)
    return unattacked

# receives some cb
# returns the grounded extension of the argument framework for that cb 
def find_grounded(cb):    
    increasing = True
    old_len_grounded = 0
    while increasing:
        grounded = get_unattacked(cb)
        if len(grounded) > old_len_grounded:
            for g in grounded:
                attacked = are_attacked(g, cb)
            cb = [i for i in cb if i not in attacked]
            old_len_grounded = len(grounded)
        else:
            increasing = False 
            
    return grounded
            
# receives a new case and some cb
# returns a prediction for the new case
def make_prediction(new_case, cb):
    removals = []
    for case in cb.cases:
        for feature in case.fs.keys():
            if feature not in new_case.fs.keys():
                removals.append(case)
                break
                
    unattacked = [i for i in cb.cases if i not in removals]

    default = Case('default',{},0)
    unattacked.append(default)
    
    grounded = find_grounded(unattacked)

    if default in grounded:
        return default.outcome
    else:
        return default.opposite_outcome
    

In [7]:
# Example code to run a test
def run_aa_cbr():

    cb = create_cb()
    test, train = cb.split(0.1)
    
    correct = 0
    for t in test.cases:
        pre = make_prediction(t, train)
        if pre == t.outcome:
            correct += 1
    print(correct/test.length)
    
run_aa_cbr()

1.0
